In [379]:
# Importing necessary libraries
import pandas as pd 
import numpy as np

In [380]:
# Loading in CSV files as dataframes and previewing the first 5 rows of the all_nba_teams dataframe
all_nba_teams = pd.read_csv('/Users/jackewings/Documents/Projects/All-NBA-Prediction/data/raw/End of Season Teams.csv')
per_game = pd.read_csv('/Users/jackewings/Documents/Projects/All-NBA-Prediction/data/raw/Player Per Game.csv')
advanced = pd.read_csv('/Users/jackewings/Documents/Projects/All-NBA-Prediction/data/raw/Advanced.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

all_nba_teams.head()

,season,lg,type,number_tm,player,player_id,position
0,2024,NBA,All-Defense,1st,Rudy Gobert,goberru01,NaN
1,2024,NBA,All-Defense,1st,Victor Wembanyama,wembavi01,NaN
2,2024,NBA,All-Defense,1st,Bam Adebayo,adebaba01,NaN
3,2024,NBA,All-Defense,1st,Herbert Jones,joneshe01,NaN
4,2024,NBA,All-Defense,1st,Anthony Davis,davisan02,NaN


In [381]:
# Filtering to only include seasons after 2019 and to only include All-NBA awards
df_20s_all_nba = all_nba_teams[(all_nba_teams['season'] >= 2020) & (all_nba_teams['type'] == 'All-NBA')].reset_index()

df_20s_all_nba.head()

,index,season,lg,type,number_tm,player,player_id,position
0,10,2024,NBA,All-NBA,1st,Nikola Jokić,jokicni01,C
1,11,2024,NBA,All-NBA,1st,Giannis Antetokounmpo,antetgi01,F
2,12,2024,NBA,All-NBA,1st,Jayson Tatum,tatumja01,F
3,13,2024,NBA,All-NBA,1st,Luka Dončić,doncilu01,G
4,14,2024,NBA,All-NBA,1st,Shai Gilgeous-Alexander,gilgesh01,G


In [382]:
# Subsetting to only include needed columns and creating a new numeric column which is 1 if the award type was All-NBA
df_20s_all_nba = df_20s_all_nba[['player_id', 'player', 'season']]

df_20s_all_nba['all_nba'] = 1

df_20s_all_nba.head()

,player_id,player,season,all_nba
0,jokicni01,Nikola Jokić,2024,1
1,antetgi01,Giannis Antetokounmpo,2024,1
2,tatumja01,Jayson Tatum,2024,1
3,doncilu01,Luka Dončić,2024,1
4,gilgesh01,Shai Gilgeous-Alexander,2024,1


In [383]:
# Previewing the first 5 rows of the Player per Game dataframe
per_game.head()

,season,lg,player,player_id,age,team,pos,g,gs,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,x3p_percent,x2p_per_game,x2pa_per_game,x2p_percent,e_fg_percent,ft_per_game,fta_per_game,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,2025,NBA,Precious Achiuwa,achiupr01,25.0,NYK,C,57,10.0,20.5,2.9,5.7,0.502,0.2,0.6,0.278,2.7,5.1,0.529,0.517,0.7,1.2,0.594,1.8,3.8,5.6,1.0,0.8,0.7,0.8,1.4,6.6
1,2025,NBA,Steven Adams,adamsst01,31.0,HOU,C,58,3.0,13.7,1.6,2.9,0.545,0.0,0.0,0.000,1.6,2.8,0.552,0.545,0.7,1.6,0.462,2.9,2.8,5.6,1.1,0.4,0.5,0.9,1.0,3.9
2,2025,NBA,Bam Adebayo,adebaba01,27.0,MIA,C,78,78.0,34.3,6.9,14.3,0.485,1.0,2.8,0.357,5.9,11.4,0.517,0.521,3.2,4.2,0.765,2.4,7.2,9.6,4.3,1.3,0.7,2.1,2.1,18.1
3,2025,NBA,Ochai Agbaji,agbajoc01,24.0,TOR,SG,64,45.0,27.2,4.2,8.3,0.498,1.6,4.0,0.399,2.6,4.4,0.587,0.593,0.5,0.8,0.708,1.0,2.8,3.8,1.5,0.9,0.5,0.8,2.0,10.4
4,2025,NBA,Santi Aldama,aldamsa01,24.0,MEM,PF,65,16.0,25.5,4.8,10.0,0.483,1.8,5.0,0.368,3.0,5.0,0.599,0.576,1.0,1.4,0.691,1.4,5.0,6.4,2.9,0.8,0.4,1.1,1.2,12.5


In [384]:
# Filtering for specific years and removing ineligible players
df_per_game_20s = per_game[(per_game['season'] >= 2020) & (per_game['season'] != 2025) & (per_game['g'] >= 45)]

# Subsetting for the needed columns
df_per_game_20s = df_per_game_20s[['season', 'player_id', 'age', 'x2p_percent', 'x3p_percent', 'ft_percent', 'pts_per_game',
                                   'trb_per_game', 'ast_per_game', 'stl_per_game', 'blk_per_game']].copy()

# Grouping by player_id and getting the average stats for each player
# Needed if a player played on multiple teams 
df_per_game_20s_agg = df_per_game_20s.groupby(['player_id', 'season']).mean().round(3).reset_index()

df_per_game_20s_agg = df_per_game_20s_agg.fillna(0)

df_per_game_20s_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player_id     1561 non-null   object 
 1   season        1561 non-null   int64  
 2   age           1561 non-null   float64
 3   x2p_percent   1561 non-null   float64
 4   x3p_percent   1561 non-null   float64
 5   ft_percent    1561 non-null   float64
 6   pts_per_game  1561 non-null   float64
 7   trb_per_game  1561 non-null   float64
 8   ast_per_game  1561 non-null   float64
 9   stl_per_game  1561 non-null   float64
 10  blk_per_game  1561 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 134.3+ KB


In [385]:
# Previewing the first 5 rows of the Advanced dataframe
advanced.head()

,season,lg,player,player_id,age,team,pos,g,gs,mp,per,ts_percent,x3p_ar,f_tr,orb_percent,drb_percent,trb_percent,ast_percent,stl_percent,blk_percent,tov_percent,usg_percent,ows,dws,ws,ws_48,obpm,dbpm,bpm,vorp
0,2025,NBA,Precious Achiuwa,achiupr01,25.0,NYK,C,57,10.0,1170.0,13.6,0.530,0.110,0.211,10.0,21.0,15.5,6.2,2.0,3.3,11.2,14.9,0.7,1.7,2.3,0.095,-2.1,0.2,-1.9,0.0
1,2025,NBA,Steven Adams,adamsst01,31.0,HOU,C,58,3.0,794.0,16.6,0.541,0.012,0.557,21.8,22.0,21.9,10.9,1.3,3.2,20.6,13.6,1.0,1.3,2.3,0.137,0.0,0.3,0.3,0.5
2,2025,NBA,Bam Adebayo,adebaba01,27.0,MIA,C,78,78.0,2674.0,18.9,0.561,0.199,0.295,7.7,23.1,15.5,20.0,1.8,1.9,11.4,23.5,3.5,4.0,7.5,0.135,1.0,1.0,2.0,2.7
3,2025,NBA,Ochai Agbaji,agbajoc01,24.0,TOR,SG,64,45.0,1739.0,12.0,0.601,0.474,0.090,3.7,11.4,7.5,7.9,1.6,1.6,8.9,14.6,1.6,1.5,3.1,0.087,-0.9,0.2,-0.7,0.5
4,2025,NBA,Santi Aldama,aldamsa01,24.0,MEM,PF,65,16.0,1660.0,16.8,0.588,0.503,0.145,6.1,20.3,13.3,15.2,1.5,1.6,9.3,18.5,2.9,2.0,5.0,0.144,2.5,0.7,3.2,2.2


In [386]:
# Filtering for specific years and removing ineligible players
df_advanced_20s = advanced[(advanced['season'] >= 2020) & (advanced['season'] < 2025) & (per_game['g'] >= 45)]

# Subsetting for the needed columns
df_advanced_20s = df_advanced_20s[['player_id', 'player', 'season', 'x3p_ar', 'f_tr', 'orb_percent', 'drb_percent', 'usg_percent', 'ows', 'dws',
                                   'obpm', 'dbpm']]

# Grouping by player_id and getting the average stats for each player
# Needed if a player played on multiple teams 
df_means = df_advanced_20s.groupby(['player_id', 'season']).mean(numeric_only=True).round(3).reset_index()

df_names = df_advanced_20s.groupby('player_id')['player'].first().reset_index()

df_advanced_20s_agg = df_means.merge(df_names, on = 'player_id', how = 'left')


df_advanced_20s_agg.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player_id    1561 non-null   object 
 1   season       1561 non-null   int64  
 2   x3p_ar       1561 non-null   float64
 3   f_tr         1561 non-null   float64
 4   orb_percent  1561 non-null   float64
 5   drb_percent  1561 non-null   float64
 6   usg_percent  1561 non-null   float64
 7   ows          1561 non-null   float64
 8   dws          1561 non-null   float64
 9   obpm         1561 non-null   float64
 10  dbpm         1561 non-null   float64
 11  player       1561 non-null   object 
dtypes: float64(9), int64(1), object(2)
memory usage: 146.5+ KB


In [387]:
# Merging the cleaned per game and advanced datasets into a new dataframe df_all and updating datatypes
player_stats_all_nba = df_per_game_20s_agg.merge(df_advanced_20s_agg, on = ['player_id', 'season'], how = 'inner')

df_all['season'] = player_stats_all_nba['season'].astype(int)
df_all['age'] = player_stats_all_nba['age'].astype(int)

player_stats_all_nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player_id     1561 non-null   object 
 1   season        1561 non-null   int64  
 2   age           1561 non-null   float64
 3   x2p_percent   1561 non-null   float64
 4   x3p_percent   1561 non-null   float64
 5   ft_percent    1561 non-null   float64
 6   pts_per_game  1561 non-null   float64
 7   trb_per_game  1561 non-null   float64
 8   ast_per_game  1561 non-null   float64
 9   stl_per_game  1561 non-null   float64
 10  blk_per_game  1561 non-null   float64
 11  x3p_ar        1561 non-null   float64
 12  f_tr          1561 non-null   float64
 13  orb_percent   1561 non-null   float64
 14  drb_percent   1561 non-null   float64
 15  usg_percent   1561 non-null   float64
 16  ows           1561 non-null   float64
 17  dws           1561 non-null   float64
 18  obpm          1561 non-null 

In [388]:
# Merging the all_nba dataframe with df_all and creating the all_nba column in the final dataframe
all_nba_ref = df_20s_all_nba[['player_id', 'season']].drop_duplicates()
all_nba_ref['all_nba'] = 1


player_stats_all_nba = player_stats_all_nba.merge(all_nba_ref, on = ['player_id', 'season'], how = 'left')


player_stats_all_nba['all_nba'] = player_stats_all_nba['all_nba'].fillna(0).astype(int)

In [389]:
# Confirming that the number of All-NBA players for each season is 15
player_stats_all_nba.groupby('season')['all_nba'].sum()

season
2020    15
2021    15
2022    15
2023    15
2024    15
Name: all_nba, dtype: int64

In [390]:
# Sorting the dataframe by season, player_id
player_stats_all_nba = player_stats_all_nba.sort_values(by = ['season', 'player_id'])

player_stats_all_nba.head()

,player_id,season,age,x2p_percent,x3p_percent,ft_percent,pts_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,x3p_ar,f_tr,orb_percent,drb_percent,usg_percent,ows,dws,obpm,dbpm,player,all_nba
4,adamsst01,2020,26.0,0.594,0.333,0.582,10.9,9.3,2.3,0.8,1.1,0.006,0.421,14.0,24.0,17.3,3.8,2.7,1.9,1.1,Steven Adams,0
7,adebaba01,2020,22.0,0.564,0.143,0.691,15.9,10.2,5.1,1.1,1.3,0.018,0.484,8.5,24.9,21.2,4.6,3.9,1.4,2.0,Bam Adebayo,0
16,aldrila01,2020,34.0,0.519,0.389,0.827,18.9,7.4,2.4,0.7,1.6,0.198,0.241,6.3,17.8,23.4,3.0,1.4,1.8,-0.5,LaMarcus Aldridge,0
18,alexani01,2020,21.0,0.391,0.346,0.676,5.7,1.8,1.9,0.4,0.2,0.500,0.139,1.6,13.5,23.3,-0.7,0.4,-3.2,-1.4,Nickeil Alexander-Walker,0
27,allenja01,2020,21.0,0.658,0.000,0.633,11.1,9.6,1.6,0.6,1.3,0.013,0.581,12.3,24.9,14.9,5.2,2.9,1.4,1.0,Jarrett Allen,0


In [391]:
player_stats_all_nba = player_stats_all_nba[['player_id', 'player', 'season', 'age', 'pts_per_game', 'trb_per_game', 
                                             'ast_per_game', 'stl_per_game', 'blk_per_game', 'x2p_percent', 'x3p_percent', 
                                             'ft_percent', 'x3p_ar', 'f_tr', 'usg_percent', 'orb_percent', 'drb_percent', 
                                             'ows', 'dws', 'obpm', 'dbpm', 'all_nba']]

player_stats_all_nba.head()

,player_id,player,season,age,pts_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,x2p_percent,x3p_percent,ft_percent,x3p_ar,f_tr,usg_percent,orb_percent,drb_percent,ows,dws,obpm,dbpm,all_nba
4,adamsst01,Steven Adams,2020,26.0,10.9,9.3,2.3,0.8,1.1,0.594,0.333,0.582,0.006,0.421,17.3,14.0,24.0,3.8,2.7,1.9,1.1,0
7,adebaba01,Bam Adebayo,2020,22.0,15.9,10.2,5.1,1.1,1.3,0.564,0.143,0.691,0.018,0.484,21.2,8.5,24.9,4.6,3.9,1.4,2.0,0
16,aldrila01,LaMarcus Aldridge,2020,34.0,18.9,7.4,2.4,0.7,1.6,0.519,0.389,0.827,0.198,0.241,23.4,6.3,17.8,3.0,1.4,1.8,-0.5,0
18,alexani01,Nickeil Alexander-Walker,2020,21.0,5.7,1.8,1.9,0.4,0.2,0.391,0.346,0.676,0.500,0.139,23.3,1.6,13.5,-0.7,0.4,-3.2,-1.4,0
27,allenja01,Jarrett Allen,2020,21.0,11.1,9.6,1.6,0.6,1.3,0.658,0.000,0.633,0.013,0.581,14.9,12.3,24.9,5.2,2.9,1.4,1.0,0


In [392]:
player_stats_all_nba.to_csv(
    '/Users/jackewings/Documents/Projects/All-NBA-Prediction/data/processed/player_stats_all_nba.csv', index = False
)